In [1]:
import numpy as np

from numpy.testing import assert_allclose

# QR decomposition

Any real $m \times n$ matrix can be decomposed into 
$$\mathbf{A} = \mathbf{Q} \cdot \mathbf{R}, $$
where $\mathbf{Q}$ is an $m \times m$ orthogonal matrix ($\mathbf{Q}^T \mathbf{Q} = \mathbf{1}$, here $\mathbf{1}$ is an identity matrix) and $\mathbf{R}$ is an $m \times n$ upper triangular matrix.

If $m > n$
$$\mathbf{A} = \mathbf{Q} \begin{pmatrix}
            \mathbf{R_1} \\
            \mathbf{0}
        \end{pmatrix} = 
        \begin{pmatrix}
            \mathbf{Q_1} & \mathbf{Q_2}\\ 
        \end{pmatrix}
      \begin{pmatrix}
            \mathbf{R_1} \\
            \mathbf{0}
       \end{pmatrix} = \mathbf{Q_1} \mathbf{R_1},$$
where $\mathbf{R_1}$ is $m \times m$ upper triangular, $\mathbf{Q_1}$ is $m \times n$ matrix and has orthonormal columns. This operation is called *thin QR* (*reduced*) factorization. 

If columns of $\mathbf{A}$ are linearly independent, then
1. The thin factorization $\mathbf{A} = \mathbf{Q_1} \mathbf{R_1}$ is unique;
2. Diagonal elements of $\mathbf{R_1}$ are positive.

One way to reduce $\mathbf{A}$ to $\mathbf{R}$ and construct $\mathbf{Q}$ is to implement Householder reflection algorithm column by column.

## Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

In [2]:
from IPython.display import Image
Image("pic1.png")

TypeError: a bytes-like object is required, not 'str'

TypeError: a bytes-like object is required, not 'str'

<IPython.core.display.Image object>

The matrix of the Householder transformation is
$$
\mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T.
$$
Note that $H$ is symmetric and orthogonal.

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{||\mathbf{x} - \mathbf{y}||}
$$

In [3]:
Image("pic2.png")

TypeError: a bytes-like object is required, not 'str'

TypeError: a bytes-like object is required, not 'str'

<IPython.core.display.Image object>

Write a function which rotates a given vector, 
$$\mathbf{x} = 
      \begin{pmatrix}
            x_1 \\
            x_2 \\
            \vdots \\
            x_n
       \end{pmatrix} \;\; \mbox{into} \;\; \mathbf{y} = 
      \begin{pmatrix}
            ||\mathbf{x}|| \\
            0 \\
            \vdots \\
            0
       \end{pmatrix}$$ 
using a Householder transformation.

In [1]:
import math
import numpy as np
def vector_norm(vec):
    result = 0
    for k in range (len(vec)):
        result = result + vec[k] * vec[k]
    return math.sqrt(result)

In [2]:
def householder(vec):
    """Construct a Householder reflection to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Transformed vector, with ``outvec[1:]==0`` and ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Orthogonal matrix of the Householder reflection
    """
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    
    n = len(vec)
    y = np.asarray(([0.0]*n),dtype=float)
    y[0] = vector_norm( vec)
    v1 = vec - y
    v1 /= vector_norm(v1)
    I = np.eye(n)
    V_trans = np.atleast_2d(v1)
    H = I - 2 * (V_trans.T @ V_trans)
    vec_new = H @ vec
    return vec_new, H

In [3]:
v = np.array([1., 2., 3.])
v1, h = householder(v)

np.testing.assert_allclose(np.dot(h, v1), v)
np.testing.assert_allclose(np.dot(h, v), v1)

Test your function using tests below:

In [4]:
# Test I.2

rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

np.testing.assert_allclose(np.dot(h, v1), vec)

In [66]:
n = h.shape[0]
s = 0
for i in range(n):
    s+= h[i, i]

In [67]:
s

5.000000000000001

In [69]:
h

array([[ 0.13572382,  0.44086895,  0.31020391,  0.55655897,  0.55274437,
         0.19317782,  0.19592154],
       [ 0.44086895,  0.77511189, -0.15823561, -0.28390181, -0.28195597,
        -0.09854038, -0.09993996],
       [ 0.31020391, -0.15823561,  0.88866237, -0.19975879, -0.19838967,
        -0.06933491, -0.07031968],
       [ 0.55655897, -0.28390181, -0.19975879,  0.64159849, -0.35594506,
        -0.12439872, -0.12616556],
       [ 0.55274437, -0.28195597, -0.19838967, -0.35594506,  0.64649455,
        -0.1235461 , -0.12530083],
       [ 0.19317782, -0.09854038, -0.06933491, -0.12439872, -0.1235461 ,
         0.95682205, -0.04379121],
       [ 0.19592154, -0.09993996, -0.07031968, -0.12616556, -0.12530083,
        -0.04379121,  0.95558683]])

See how many tests (there are 2 of them - I.1 and I.2) your function passed. Sum diagonal elements in matrix `h`. You will need this for Google Form.

## Part II. Compute the $\mathrm{QR}$ decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $\mathrm{QR}$ decomposition of $\mathbf{A}$. 

Write a function, which receives a recangular matrix, $A$, and returns the Q and R factors of the $QR$ factorization of $A$.

In [58]:
def get_column(a, i, j):
    v = np.zeros(a.shape[1] - j)
    for k in range(j, a.shape[1]):
        v[k - j] = a[i, k]
    return v

In [62]:
from math import copysign, hypot
import numpy as np
from math import sqrt
from pprint import pprint

def qr_decomp(matrix):
    """Compute the QR decomposition of a matrix.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    q : ndarray, shape(m, m)
        The orthogonal matrix
    r : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    width = matrix.shape[0]
    height = matrix.shape[1]
    r = np.zeros((width, height))
    h_mult = np.zeros((width, height))
    for i in range(width):
        v = get_column(a, i, i)
        hi = householder(v)
        if (i == 0):
            h_mult = hi
            print(hi)
        else:
            print(hi)
            h_mult = np.dot(h_mult, hi)
    r = np.dot(h_mult, a)
    q = np.dot(a * r.T)
    return q, r
    

In [63]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)

In [64]:
# Test II.1

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
v = get_column(a, 0, 1)
q, r = qr_decomp(a)

# test that Q is indeed orthogonal
np.testing.assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
np.testing.assert_allclose(np.dot(q, r), a)

(array([ 0.78441354, -0.        ,  0.        ]), array([[ 0.24415623,  0.79308776,  0.55803185],
       [ 0.79308776,  0.16783307, -0.58552871],
       [ 0.55803185, -0.58552871,  0.5880107 ]]))
(array([ 0.82623785, -0.        ]), array([[ 0.9440088 ,  0.32992026],
       [ 0.32992026, -0.9440088 ]]))


ValueError: could not broadcast input array from shape (3,3) into shape (3)

Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions).

In [ ]:
# Test II.2

from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)
assert_allclose(qq, q)
assert_allclose(rr, r)

Did `q` and `r` agree with `qq` and `rr`? If not, explain why. You will need this for Google Form.

## Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).

Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. 

Write a second function, which uses reflection vectors to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests. 

In [ ]:
def qr_decomp_new(a):
    
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    
    # ... ENTER YOUR CODE HERE ...
    
    return q, r

In [ ]:
# Test III.1

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q_new, r_new = qr_decomp_new(a)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q_new, q_new.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q_new, r_new), a)

And again, compare your decompositions to the library function.

In [ ]:
# Test III.2

assert_allclose(qq, q_new)
assert_allclose(rr, r_new)

Sum all elements in matrix `q_new` and `r_new` separately. You will need this for Google Form.